Modes of correlation: Model (oracle, personal), Granularity (recipe level, user level, recipe flat, user flat, all recipes for all users), Surprise (95th percentile, max), Method of correlation (spearman, pearson)

Libraries


In [1]:
import os, pickle, numpy as np
from GloVex.evaluate_personalised import survey_reader
from scipy.stats import pearsonr, spearmanr
cwd = os.getcwd()

Get the oracle surprise estimates by the model

In [2]:
oracle_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/oracle_suprise_estimates.pickle'
oracle_suprise_estimates_dict = pickle.load(open(oracle_suprise_estimates_pickle_fn, 'rb'))

Get the oracle's surprise estimates (95th percentile and max score)

In [3]:
# Initialize arrays
oracle_surp_estimates_95perc = []
oracle_surp_estimates_max = []
# Iterate over the dict
for each_recipe in oracle_suprise_estimates_dict:
	# Get all of the ingredients combinations' surprise estimates
	oracle_ingr_surp_estimates = [each_ingr_comb[2] for each_ingr_comb in oracle_suprise_estimates_dict[each_recipe]['surprise_cuisine']]
	# Append the 95th_percentile surprise estimate
	oracle_surp_estimates_95perc.append(oracle_suprise_estimates_dict[each_recipe]['95th_percentile'])
	# Append the max score surprise estimate
	oracle_surp_estimates_max.append(max(oracle_ingr_surp_estimates))
# oracle_surp_estimates_95perc
# oracle_surp_estimates_max

Get the personal surprise estimates by the model

In [4]:
user_suprise_estimates_pickle_fn = cwd + '/GloVex/results/new/user_suprise_estimates.pickle'
user_suprise_estimates_dict = pickle.load(open(user_suprise_estimates_pickle_fn,'rb'))

Get the 95th percentils and max surprise estimates

In [5]:
# Initialize surprise estimates for 95th percentile and max score
personalized_surp_estimates_95perc = []
personalized_surp_estimates_max = []
# Iterate over the user surprise estimeates dict
for each_user in user_suprise_estimates_dict:
	# print 'each_user', each_user, user_suprise_estimates_dict[each_user]
	# Initialize the inner arrays for 95th percentile and max score
	users_surp_estimates_95perc = []
	users_surp_estimates_max = []
	# Iterate over the surprises for each user's surprise estimates
	for recipe_idx, each_recipe in enumerate(user_suprise_estimates_dict[each_user]['recipes_surp']):
		# print recipe_idx, user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile']
		# Get all of the ingredients combinations' surprise estimates
		ingr_comb_suprise_arr = [each_ingr_comb[2] for each_ingr_comb in user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['surprise_cuisine']]
		# print ingr_comb_suprise_arr
		# print max(ingr_comb_suprise_arr)
		# Append the 95th percentile for the surprise estimates
		users_surp_estimates_95perc.append(user_suprise_estimates_dict[each_user]['recipes_surp'][recipe_idx]['95th_percentile'])
		# Append the max score for the surprise estimates
		users_surp_estimates_max.append(max(ingr_comb_suprise_arr))
	# Append the user's surprise estimates to the personalized main 2D array (95th percentile and max score)
	personalized_surp_estimates_95perc.append(users_surp_estimates_95perc)
	personalized_surp_estimates_max.append(users_surp_estimates_max)
# personalized_surp_estimates_95perc
# personalized_surp_estimates_max


Transpose for recipe level comparison

In [6]:
personalized_surp_estimates_recipelvl_95perc = list(np.transpose(personalized_surp_estimates_95perc))
personalized_surp_estimates_recipelvl_max = list(np.transpose(personalized_surp_estimates_max))

Flatten the arrays in both dimnessions

In [7]:
personalized_surp_estimates_userflat_95perc = [item for sublist in personalized_surp_estimates_95perc for item in sublist]
personalized_surp_estimates_userflat_max = [item for sublist in personalized_surp_estimates_max for item in sublist]
personalized_surp_estimates_recipeflat_95perc = [item for sublist in personalized_surp_estimates_recipelvl_95perc for item in sublist]
personalized_surp_estimates_recipeflat_max = [item for sublist in personalized_surp_estimates_recipelvl_max for item in sublist]

Get the surprises of the users in the survey

In [8]:
survey_fp = '/Volumes/GoogleDrive/My Drive/Live/PQE/Omar/personalized-surprise/qchef_surveydata'
user_fam_scaled_arr, users_surp_ratings_arr = survey_reader(survey_fp)

Transpose for recipe level comparison

In [9]:
users_surp_ratings_userlvl = users_surp_ratings_arr
users_surp_ratings_recipelvl = list(np.transpose(users_surp_ratings_arr))

Flatten the arrays in both dimenssions

In [10]:
users_surp_ratings_user_flat = [item for sublist in users_surp_ratings_userlvl for item in sublist]
users_surp_ratings_recipe_flat = [item for sublist in list(np.transpose(users_surp_ratings_arr)) for item in sublist]

Mode: Oracle + Personalized, per user (default for oracle), 95th percentile

In [12]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_95perc, users_surp_ratings_userlvl)):
	print user_idx
	print each_personal_score
	print each_users_rating
	print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates_95perc, each_users_rating)
	print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates_95perc, each_users_rating)
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0
[3.4992518111744015, 1.820851219042163, 0.7769672304116093, 0.7493311274020704, 0.6586530676765014, 0.25178225605472326, 0.028929721010827643, -0.34728576461298233, -0.4468323932292995, -0.8863879953910269]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.30813875924572515, 0.38638381485150064)
oracle_surp_estimates: pearsonr (0.22543514207124077, 0.5311675107950075)
each_personal_score: spearmanr (-0.4785984984029348, 0.16171268487214444)
each_personal_score: pearsonr (-0.6849534673651423, 0.02884843147663722)
1
[3.8311755317516774, 2.5636580994075437, 1.49734122161952, 1.3938903613409888, 1.3311161275546537, 0.7757765059679845, 0.5436363806717961, 0.1339915324871328, 0.10003276519671964, -0.24937998554519694]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2
[3.7035750118237103, 2.42695464967051, 1.269551777990076

Mode: Oracle + Personalized, per user (default for oracle), max score

In [13]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_max, users_surp_ratings_userlvl)):
	print user_idx
	print each_personal_score
	print each_users_rating
	print 'oracle_surp_estimates: spearmanr', spearmanr(oracle_surp_estimates_max, each_users_rating)
	print 'oracle_surp_estimates: pearsonr', pearsonr(oracle_surp_estimates_max, each_users_rating)
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0
[4.407839848709313, 2.353106981887523, 1.4716768665424258, 3.8059580332580722, 0.7603070766366502, 0.8105135465639222, 0.48790048028225025, 0.35901522166489186, -0.03543186751564214, -0.707503280261354]
[1, 1, 4, 4, 1, 4, 5, 4, 3, 4]
oracle_surp_estimates: spearmanr (0.36058790975563576, 0.3060212830567241)
oracle_surp_estimates: pearsonr (0.2882286617927404, 0.4193198598657479)
each_personal_score: spearmanr (-0.3474756221281581, 0.3252140444837802)
each_personal_score: pearsonr (-0.42500349330409515, 0.22081690920088673)
1
[4.546031735017476, 2.6834683624340157, 2.1116708950495306, 4.57327267745409, 1.5656208905644358, 1.3540113489072998, 1.021739912141993, 0.4826758817105218, 1.0647990501959337, -0.04521917008970724]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
oracle_surp_estimates: spearmanr (nan, nan)
oracle_surp_estimates: pearsonr (nan, 1.0)
each_personal_score: spearmanr (nan, nan)
each_personal_score: pearsonr (nan, 1.0)
2
[4.363872028077286, 2.569457793870565, 4.4349466540026485, 2.0872

Mode: Personalized, per recipes, 95th percentile

In [14]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl_95perc, users_surp_ratings_recipelvl)):
	print user_idx
	print each_personal_score
	print each_users_rating
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0
[3.49925181 3.83117553 3.70357501 3.84503235 3.56308545 3.55365942
 3.52479258 2.78596854 3.94599764 2.78798575 2.97942869 3.50245771
 3.31358553 3.25183134 3.35378923 3.19129138 3.07448799 3.38147875
 3.71717575 3.95641533 3.80879298 3.56271756 3.4684692  3.37236608
 3.44662044 3.56740045 3.44886856 3.48936945 3.38091749 3.3745709
 3.95349189 2.98370526 3.00932009 2.82581528 3.12910248 3.40617894
 3.90665261 3.92156704 3.83901396 3.56271756 3.604278   3.80742229
 3.93550914 3.34274754 3.95641533 2.57421132 3.95641533 3.28263522
 3.03635384 3.32313969 3.29408604 3.18343797 3.10122902 3.503964
 3.37331352 3.53672889 3.52059868 3.54345722 3.85537237 3.27318423
 2.61906749 3.46205603 3.51190203 2.45794335 3.71643483 3.04938486
 2.53610502 3.12910248 2.78464555 3.56271756 1.5423782  3.65153175
 3.45648711 3.3537925  3.37236608 3.24315384 0.90182805 3.56271756
 3.86543912 2.93232288 3.26659173 3.17699623 3.56271756 3.33605303
 3.02311096 3.51636841]
[1 1 2 1 1 1 1 1 1 1 1 1 3 4 1 1 1 1 1 

Mode: Personalized, per recipes, max score

In [15]:
for user_idx, (each_personal_score, each_users_rating) in enumerate(zip(personalized_surp_estimates_recipelvl_max, users_surp_ratings_recipelvl)):
	print user_idx
	print each_personal_score
	print each_users_rating
	print 'each_personal_score: spearmanr', spearmanr(each_personal_score, each_users_rating)
	print 'each_personal_score: pearsonr', pearsonr(each_personal_score, each_users_rating)

0
[4.40783985 4.54603174 4.36387203 4.50962086 4.3727381  4.57150854
 4.14566528 3.95161408 4.53781601 3.78893535 3.77461302 4.41943058
 4.26066996 4.19234764 4.22578697 4.47621929 4.3298649  4.26457282
 4.65553435 4.54386538 4.66207726 4.21755481 4.18458341 4.71684233
 4.22329009 4.82715323 4.28590569 4.56863993 4.37384228 4.38978259
 4.52423367 4.22819531 4.36039932 3.64912611 4.35493604 4.21285704
 4.56455517 4.60652968 4.79618733 4.21755481 4.43917096 4.67570889
 4.53174396 4.57622031 4.54386538 3.67483417 4.54386538 3.9079178
 4.34813021 4.25749493 3.71442468 4.07447656 4.38319589 4.30567248
 4.39544756 4.1632352  4.22795441 4.24339985 4.57999564 4.28828954
 3.05531839 4.12544086 4.17448773 3.22259681 4.07462794 3.83968746
 3.02980969 4.35493604 3.31808506 4.21755481 2.27596516 4.43887828
 4.4844223  4.32166726 4.71684233 4.15751115 2.15341087 4.21755481
 4.59597658 4.04068489 4.09127901 4.16184389 4.21755481 3.84223176
 3.85145747 4.16443243]
[1 1 2 1 1 1 1 1 1 1 1 1 3 4 1 1 1 1 

Modes: Personalized, all users, user flat, 95th percentile/max score

In [16]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_95perc, users_surp_ratings_user_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_userflat_max, users_surp_ratings_user_flat)

95th percentile
spearmanr (-0.30481904983015, 5.988071696087487e-20)
pearsonr (-0.31737659984687483, 1.4073000266857855e-21)
Max score
spearmanr (-0.27003791143270794, 7.773824208932607e-16)
pearsonr (-0.24738284583778478, 1.8555203868603025e-13)


Modes: Personalized, all recipes, recipe flat, 95th percentile/max score

In [17]:
print '95th percentile'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_95perc, users_surp_ratings_recipe_flat)
print 'Max score'
print 'spearmanr', spearmanr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)
print 'pearsonr', pearsonr(personalized_surp_estimates_recipeflat_max, users_surp_ratings_recipe_flat)

95th percentile
spearmanr (-0.30481904983015, 5.988071696087487e-20)
pearsonr (-0.31737659984687483, 1.4073000266857855e-21)
Max score
spearmanr (-0.27003791143270794, 7.773824208932607e-16)
pearsonr (-0.24738284583778472, 1.8555203868603025e-13)
